# Wave Interference

The plotted function is the superimposition of $n$ 2d-sine  functions centered in $(0,Y_i)$:
$$f(x,y)=\sin\big(p\cdot\sqrt{x^2+(y-Y_i)^2}\big)$$

The parameters that can be changed are:

-- $n$ the number of sources (start with 2)

-- $d$ the distance between sources

-- $p$ the periodicity. Higher periodicity means shorter wavelength. $p =2\pi / \lambda$. Note that in electron microscopy the usual convention is to define the wave vector as $k=1/\lambda$. 

-- $f$ the field of view.

One sees the directions of space where intereferences are constructive / destructive, with the angle between the outgoing beams increasing with decreasing $p$, and decreasing source spacing.

## Hints for the use

Start with $n=2$. 

Press on "run interact" to initiate the plotting after changing the parameters. 

With many sources, the intereference patterns close to the source is quite intricated. One needs to go to the far field (increase $f$ the field of view) to see the formation of the outgoing beams.

With more sources (try up to 10 sources), the outgoing beams are getting thinner.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = (16,16)


In [ ]:
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook, push_notebook
from bokeh.palettes import Viridis256
output_notebook()

In [ ]:
# n is the number of sources
# d distance between sources
# p changes periodicity
# f field of view

def waveplot2d(n,d,p,f):
    ###################
    #Calculation part #
    ###################
    
    #2D grid definition
    x=np.linspace(0,f,2001)
    y=np.linspace(-f/2,f/2,2001)
    X, Y = np.meshgrid(x, y)
    
    #Layout of the sources
    c=np.linspace(-(n-1)*d/2,(n-1)*d/2,n)
    c0=np.linspace(0,0,n)
    print(c)
    
    #Far field distance
    far=4*(d*n)**2/p
    
    #Calculation of the wave equations
    Z=0*X+0*Y
    Z_far=0*y
    for i in c:
        #2D representation
        Zi=(np.sin(p*np.sqrt(X**2+(Y-i)**2)))**1
        Z=Z+Zi
        #Far field
        Zi_far=(np.sin(p*np.sqrt(far**2+(y-i)**2)))**1
        Z_far+=Zi_far
    int_Z_far=np.power(Z_far,2)
    
    #############
    #Bokeh part #
    #############
    
    #2D map
    tooltips=[("value","@image"),("measured intensity","@intensity"),("x", "$x"),("y", "$y")]
    dict_Z={"image":[Z],"intensity":[np.power(Z,2)],"x":[0],"y":[-f/2]}
    p=figure(tools="hover,pan,wheel_zoom,box_zoom,reset",
             tooltips=tooltips,
             title="2D map of the wave interference between {} sources".format(n))
    p.image(source=dict_Z,image="image",x="x",y="y",dw=f,dh=f,palette=Viridis256)
    
    #Far field line
    pv = figure(toolbar_location=None,
                plot_width=200, plot_height=p.plot_height,
                x_range=(min(int_Z_far)-0.05*abs(min(int_Z_far)),max(int_Z_far)+0.05*abs(max(int_Z_far))),
                y_range=p.y_range,
                min_border=10,
                y_axis_location="right",
                title="intensity at {:2f} distance".format(far))
    
    #Comment out this out to plot the wave in the far field
    #pv.line(Z_far,y,legend_label='value', color='navy')
    
    pv.line(int_Z_far,y,legend_label='intensity', color='red')
    
    #Comment out this line to display the legend
    #pv.legend.location = "top_left"
    
    #Layout and plotting
    layout = gridplot([[p, pv], [None, None]], merge_tools=False)
    show(layout,notebook_handle=True)
    push_notebook()

    
    
#Interactive 
interact_manual(waveplot2d,n=(1,10,1),d=(2,20,1),p=(1,5,1),f=(50,900,50))